# Neural Network test

In [14]:
import nn
import numpy as np
from numpy.random import uniform, randint
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [43]:
np.random.seed(0)

data = pd.DataFrame()

X_LOW = -5
X_HIGH = 5
N_X = 100

N_samples = 1000
N_funcs = 3

data['x'] = range(N_samples)
data['x'] = data['x'].apply(lambda x: np.linspace(X_LOW, X_HIGH, N_X))


def random_func(x):
    i = randint(0, N_funcs)
    expected = [int(j == i) for j in range(N_funcs)]
    expected = np.array(expected)
    if i == 0:
        return uniform() * np.sin(uniform() * x + uniform()) + uniform(), expected
    elif i == 1:
        return uniform() * x + uniform(), expected
    elif i == 2:
        return uniform() * x ** 2 + uniform() * x + uniform(), expected

data[['food', 'expected']] = data['x'].apply(random_func).apply(pd.Series)



mean = np.mean(data['food'])
data['food'] = data['food'].apply(lambda x: x - mean)
data['food'] /= data['food'].apply(np.linalg.norm)

mean = np.mean(data['expected'])
data['expected'] = data['expected'].apply(lambda x: x - mean)
data['expected'] /= data['expected'].apply(np.linalg.norm)

data.head()

,x,food,expected
0,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.08545834464170579, -0.08167728358662518, -...","[0.8159639049934325, -0.4335197541033809, -0.3..."
1,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.22412355358448988, -0.21557238794770445, -...","[-0.4317975155367154, 0.8160347250143144, -0.3..."
2,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.16813513094403282, -0.16025232004358675, -...","[0.8159639049934325, -0.4335197541033809, -0.3..."
3,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.09131091957470834, -0.08626840519700767, -...","[0.8159639049934325, -0.4335197541033809, -0.3..."
4,"[-5.0, -4.898989898989899, -4.797979797979798,...","[-0.08923269341904429, -0.08620887492343277, -...","[0.8159639049934325, -0.4335197541033809, -0.3..."


In [44]:
TEST_SIZE = 0.2
data_train = data.sample(frac=(1-TEST_SIZE), random_state=0)
data_test = data.drop(data_train.index)

In [46]:
%load_ext autoreload
%autoreload 2

N_HIDDEN = 5
ETA = 1.e-2
ETA_LIMIT = 1.e-7
N_EPOCHS = 10
ADAPTIVE_ETA = True
ETA_FACTOR = 1.5
RANDOM_SEED = 5

ACT_FUNCS = [nn.sigmoid, nn.sigmoid]

network_shape = (N_X, N_HIDDEN, N_funcs)

neural_network = nn.NeuralNetwork(shape=network_shape, act_funcs=ACT_FUNCS,
                                  eta=ETA, eta_limit=ETA_LIMIT, adaptive_eta=ADAPTIVE_ETA, eta_factor=ETA_FACTOR,
                                  random_seed=RANDOM_SEED)

df_stats = neural_network.train(df=data_train[['food', 'expected']], n_epochs=N_EPOCHS, msg_freq=10)

df_stats


[2023-04-05 09:13:59.212946]: Currently at epoch 4/10
Average duration per epoch: 2.77 s.
Estimated time of finishing: 2023-04-05 09:14:15.822302


[2023-04-05 09:14:09.892645]: Currently at epoch 7/10
Average duration per epoch: 3.11 s.
Estimated time of finishing: 2023-04-05 09:14:19.215190

Decreasing learning rate: eta = 6.667e-03
Decreasing learning rate: eta = 4.444e-03
Decreasing learning rate: eta = 2.963e-03
Decreasing learning rate: eta = 1.975e-03
Decreasing learning rate: eta = 1.317e-03
Decreasing learning rate: eta = 8.779e-04
Decreasing learning rate: eta = 5.853e-04
Decreasing learning rate: eta = 3.902e-04
Decreasing learning rate: eta = 2.601e-04
Decreasing learning rate: eta = 1.734e-04
Decreasing learning rate: eta = 1.156e-04
Decreasing learning rate: eta = 7.707e-05
Decreasing learning rate: eta = 5.138e-05
Decreasing learning rate: eta = 3.425e-05
Decreasing learning rate: eta = 2.284e-05
Decreasing learning rate: eta = 1.522e-05
Decreasing learning rate: eta = 

,start,stop,E_mean,eta,acc_train,acc_test,est_finish
0,2023-04-05 09:13:48.140041,2023-04-05 09:13:51.884636,0.605209,0.01,0.365,None,None
1,2023-04-05 09:13:52.180118,2023-04-05 09:13:55.462372,0.595524,0.01,0.365,None,None
2,2023-04-05 09:13:55.710435,2023-04-05 09:13:58.953272,0.591782,0.01,0.365,None,None
3,2023-04-05 09:13:59.212946,2023-04-05 09:14:02.578260,0.589582,0.01,0.365,None,2023-04-05 09:14:15.822302
4,2023-04-05 09:14:02.826325,2023-04-05 09:14:06.144261,0.588128,0.01,0.365,None,2023-04-05 09:14:15.822302
5,2023-04-05 09:14:06.392326,2023-04-05 09:14:09.648601,0.587156,0.01,0.365,None,2023-04-05 09:14:15.822302
6,2023-04-05 09:14:09.892645,2023-04-05 09:14:13.178182,0.586558,0.01,0.365,None,2023-04-05 09:14:19.215190
7,2023-04-05 09:14:13.438845,2023-04-05 09:14:16.856101,0.586281,0.01,0.365,None,2023-04-05 09:14:19.215190
8,2023-04-05 09:14:17.107528,2023-04-05 09:15:48.304750,0.587529,0.0,0.365,None,2023-04-05 09:14:19.215190
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Starting training.

[2023-04-05 09:15:59.622629]: Currently at epoch 4/10
Average duration per epoch: 2.68 s.
Estimated time of finishing: 2023-04-05 09:16:15.685319


[2023-04-05 09:16:10.124806]: Currently at epoch 7/10
Average duration per epoch: 3.03 s.
Estimated time of finishing: 2023-04-05 09:16:19.215079

Decreasing learning rate: eta = 6.667e-03
Decreasing learning rate: eta = 4.444e-03
Decreasing learning rate: eta = 2.963e-03
Decreasing learning rate: eta = 1.975e-03
Decreasing learning rate: eta = 1.317e-03
Decreasing learning rate: eta = 8.779e-04


KeyboardInterrupt: 

In [47]:
fig = go.Figure([
    go.Scatter(
        x=df_stats['start'],
        y=df_stats['E_mean'],
        name='E_mean'
    ),
    go.Scatter(
        x=df_stats['start'],
        y=df_stats['acc_train'],
        name='acc_train'
    )
])

fig.show()

In [38]:
neural_network.accuracy(df=data_test[['food', 'expected']])

0.35

# Linear regression

In [150]:
px.scatter(df_stats['acc_train'].diff())